In [ ]:
import boto3

# Create an STS client in account A
sts_client = boto3.client('sts')

# Assume the role in account B
assumed_role_object = sts_client.assume_role(
    RoleArn="arn:aws:iam::7076********:role/cross-account-29970*******-role",
    RoleSessionName="cross-account-lambda-access",
    ExternalId="1234****"  # Optional, if specified in the role's trust policy
)


Current Notebook Role arn:aws:iam::29970***********:role/service-role/AmazonSageMaker-ExecutionRole-20231120T140189

If not add sts permission, the following error will appear.

An error occurred (AccessDenied) when calling the AssumeRole operation: User: arn:aws:sts::2997*******:assumed-role/AmazonSageMaker-ExecutionRole-20231120T140189/SageMaker is not authorized to perform: sts:AssumeRole on resource: arn:aws:iam::<Account-B-ID>:role/<role-name>

In [14]:
# Create a new session using the assumed role credentials
assumed_role_session = boto3.Session(
    aws_access_key_id=assumed_role_object['Credentials']['AccessKeyId'],
    aws_secret_access_key=assumed_role_object['Credentials']['SecretAccessKey'],
    aws_session_token=assumed_role_object['Credentials']['SessionToken']
)

# Create a Lambda client using the assumed role session and specify the desired region
lambda_client = assumed_role_session.client('lambda', region_name='us-east-1')

In [15]:
import json
import base64

In [16]:
def invoke_lambda_with_response_stream(function_name, payload, lambda_client):
    lambda_client = lambda_client

    response = lambda_client.invoke_with_response_stream(
        FunctionName=function_name,
        Payload=json.dumps(payload)
    )

    for event in response['EventStream']:
        if 'PayloadChunk' in event:
            chunk = event['PayloadChunk']['Payload'].decode('utf-8')
            print(chunk, end='', flush=True)
        elif 'InvokeComplete' in event:
            if 'ErrorCode' in event['InvokeComplete']:
                error_code = event['InvokeComplete']['ErrorCode']
                error_details = event['InvokeComplete']['ErrorDetails']
                print(f"\nError: {error_code} - {error_details}")
            else:
                print("\nInvocation completed successfully.")

In [17]:
# Example usage
function_name = 'sam-app-NewStreamingFunction-MvQgTK3EspsL'

messages = [
  {"role": "user", "content": "Please explain me what is Machine Learning in a simple way."},
]

payload_body={
    "anthropic_version": "bedrock-2023-05-31",
    "messages": messages, 
    "max_tokens": 300, 
    "temperature": 0.1, 
    "top_p": 0.9}

payload = {
    'modelId': 'anthropic.claude-3-sonnet-20240229-v1:0',
    'body': payload_body
}

In [19]:
invoke_lambda_with_response_stream(function_name, payload, lambda_client)

Machine Learning is a field of artificial intelligence that allows computers to learn and improve from experience without being explicitly programmed. Instead of writing code with specific instructions to accomplish a particular task, machine learning models are trained using large amounts of data.

Here's a simple analogy to understand machine learning:

Imagine you want to teach a child how to identify different types of animals. Instead of giving them a set of rules like "if it has four legs and a tail, it's a cat," you would show them many examples of different animals, along with their correct labels (e.g., pictures of cats labeled as "cat," pictures of dogs labeled as "dog," and so on).

Over time, by seeing many examples, the child would start to recognize patterns and learn to distinguish between different animals based on their characteristics, such as shape, size, color, and other features.

Similarly, in machine learning, algorithms are fed large amounts of data (like the ex